In [0]:
#to work with data stored in your google drive
from google.colab import drive 
drive.mount('/content/drive/') 



In [4]:
"if the data is in zip format then uncomment below line to unzip "
#!unzip -uq "drive/My Drive/deeplearning/trainingandtestdata.zip" -d "drive/My Drive/deeplearning/tweet140"
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import re


data = pd.read_csv('/content/drive/My Drive/deeplearning/tweet140/tweet_sentiment.csv',encoding='ISO-8859-1')
data.columns = ['sentiment','string_id','date','query','handle','text']
data = data[['sentiment','text']]
#data = data[data.sentiment != "Neutral"]
print('before randomize ', '\n',data['text'].head(5))



before randomize  
 0    is upset that he can't update his Facebook by ...
1    @Kenichan I dived many times for the ball. Man...
2      my whole body feels itchy and like its on fire 
3    @nationwideclass no, it's not behaving at all....
4                        @Kwesidei not the whole crew 
Name: text, dtype: object


In [5]:
"remove @mentions"
data = data.replace("\@[a-zA-Z]+", "",regex=True)

data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))
"Twitter sentiment data we are working on is pre-labelled. 0 is negative and 4 is positive"
print((data['sentiment']==4).sum())
print((data['sentiment']==0).sum())

"We have to randomise the data to mix up negative and positive tweets"
"randomize data"
data = data.sample(frac=1).reset_index(drop=True)
print('after randomize, lower case, removing mentions ', '\n',data['text'].head(5))



800000
799999
after randomize, lower case, removing mentions  
 0    gtotheatothegtothea gaga thats what you say  l...
1                            i love good news week    
2    how could i miss david archuletas tv guesting ...
3     ultrascreen turns out to also just be a reall...
4     nascar and asians no offense bro but that jus...
Name: text, dtype: object


In [0]:
max_features = 2000
"convert text into numbers"
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)





In [0]:
#......................................................... lstm .......
"Do note that this network is not optimised for GPU, especially recurrent_dropout. Remove that line for a GPU optimisation"
def get_model(X):
  nn = 196
  embed_dim = 128
  model = Sequential()
  model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
  #model.add(SpatialDropout1D(0.4))
  model.add(LSTM(nn, dropout=0.2))
  model.add(Dense(2,activation='softmax'))
  model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
  print(model.summary())
  return(model)

model = get_model(X=X)


In [8]:
"This Pandas function works as one-hot-encoding"
Y =  pd.get_dummies(data['sentiment']).values
Y[0]
data.iloc[0]['sentiment']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.50, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)


(799999, 40) (799999, 2)
(800000, 40) (800000, 2)


In [0]:
"We save the pre-trained model as validation accuracy improves, also useful to retrain again"
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpointer = (ModelCheckpoint(filepath="/content/drive/My Drive/deeplearning/nlp.h5", monitor='val_accuracy',
                                verbose=1, save_best_only=True))

earlystopper = EarlyStopping(monitor='val_accuracy', patience=7, verbose=1, mode='max')

k=0
trainac=[]
valac=[]
#class_weights={0:1.,1:1.}
batch_size = 332
history = model.fit(X_train, Y_train, epochs = 30, batch_size=batch_size, verbose = 2,
                    
          validation_data=(X_test,Y_test),callbacks=[earlystopper,checkpointer])


#epochs = 155 best trained afterwords overfit expect 65% accurecy

trainl=(history.history['acc']) #, label='train')
vall=(history.history['val_acc'])

df_fit=pd.DataFrame({'trainac':trainl,'valac':vall})
df_fit.plot()


In [16]:
"retrain from where you left if colab disconnects after 12 hr"
new_model = get_model(X=X)

from tensorflow.keras.models import load_model
model_path = "/content/drive/My Drive/deeplearning/nlp.h5"
n_model = load_model(model_path)
old_weights  = n_model.get_weights()
new_model.set_weights(old_weights)

batch_size = 332
history = new_model.fit(X_train, Y_train, epochs = 30, batch_size=batch_size, verbose = 2,
          validation_data=(X_test,Y_test),callbacks=[earlystopper,checkpointer])


#epochs = 155 best trained afterwords overfit expect 65% accurecy

trainl=(history.history['acc']) #, label='train')
vall=(history.history['val_acc'])

df_fit=pd.DataFrame({'trainac':trainl,'valac':vall})
df_fit.plot()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 40, 128)           256000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/30

Epoch 00001: val_accuracy improved from -inf to 0.82578, saving model to /content/drive/My Drive/deeplearning/nlp.h5
2410/2410 - 106s - loss: 0.3935 - accuracy: 0.8193 - val_loss: 0.3833 - val_accuracy: 0.8258
Epoch 2/30

Epoch 00002: val_accuracy did not improve from 0.82578
2410/2410 - 105s - loss: 0.3749 - accuracy: 0.8299 - val_loss: 0.3843 - val_accuracy

KeyboardInterrupt: ignored